# An example of early classification using RNNs

Let's start by loading data using `tslearn`.

In [9]:
import tensorflow as tf
import numpy as np
from tslearn.datasets import CachedDatasets
from model import DualOutputRNN

X_train, y_train, X_test, y_test = CachedDatasets().load_dataset("Trace")
y_train = tf.keras.utils.to_categorical(y_train)

In [10]:
tf.set_random_seed(0)
np.random.seed(0)

model = DualOutputRNN(n_classes=y_train.shape[1],
                      batch_size=10,
                      ts_size=X_train.shape[1],
                      epochs=200,
                      earliness_factor=.01,
                      lr=.001,
                      reg=.01)
model.fit(X_train, y_train)

Epoch: 0001 cost=2.352752542
Epoch: 0011 cost=1.359890079
Epoch: 0021 cost=0.899473226
Epoch: 0031 cost=0.683291930
Epoch: 0041 cost=0.712161928
Epoch: 0051 cost=0.753632867
Epoch: 0061 cost=0.742228246
Epoch: 0071 cost=0.596040660
Epoch: 0081 cost=0.463397324
Epoch: 0091 cost=0.355698921
[End of no-early training] Epoch: 0100 cost=0.436427703
Epoch: 0101 cost=1.599665475
Epoch: 0111 cost=0.844613349
Epoch: 0121 cost=0.725805366
Epoch: 0131 cost=0.609480143
Epoch: 0141 cost=0.644383860
Epoch: 0151 cost=0.531301966
Epoch: 0161 cost=0.591638231
Epoch: 0171 cost=0.527903301
Epoch: 0181 cost=0.484530118
Epoch: 0191 cost=0.476401547


In [11]:
y_pred, tau_pred = model.predict(X_test)
for yi, yi_hat, taui in zip(y_test, y_pred, tau_pred):
    print(yi, yi_hat, taui)

3 4 1
1 1 6
3 4 0
2 2 6
1 1 6
3 4 0
2 2 6
2 2 6
2 2 5
4 4 0
3 4 1
2 2 6
3 4 1
3 4 0
3 4 1
4 4 0
2 2 6
3 4 0
1 1 6
1 1 7
3 4 1
4 4 0
4 4 1
1 1 5
2 2 5
3 4 0
3 4 0
4 4 0
3 4 0
4 4 0
3 4 0
2 2 6
2 1 6
2 2 6
1 1 6
4 4 2
3 4 0
1 1 6
2 2 5
4 4 0
3 4 0
4 4 0
2 2 6
2 2 5
4 4 0
2 2 5
1 1 6
4 4 1
4 4 0
3 4 1
2 2 5
4 4 0
2 2 5
1 1 6
3 4 1
1 1 8
3 4 0
1 1 6
3 4 1
3 4 2
3 4 0
1 1 8
3 4 0
2 2 5
4 4 0
1 1 6
2 1 0
1 1 6
2 2 6
1 1 6
1 1 6
2 2 5
3 4 0
4 4 0
1 1 6
2 2 5
2 2 6
4 4 1
3 4 0
2 2 6
2 2 5
1 1 6
4 4 0
1 1 5
2 2 5
2 1 6
4 4 0
1 2 5
1 1 6
1 1 5
3 4 0
1 1 6
1 1 5
2 2 6
3 4 0
3 4 0
2 2 6
2 2 6
3 4 0
4 4 0


## Sanity tests

Some tests to check if our assumptions are correct

### 1. Insert random values at the initial timeframes.

Inserting random values at the beginning of the timeseries should delay the classification decision.
`taui` should be optimally equivalent to the values above plus the number of padded random values `pad`

This should make the taus longer since the classification-relevant information is actually comming later in the timeseries. 

In [12]:
def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

tf.set_random_seed(0)
np.random.seed(0)

# number if padded times
npad = 10

randompadded_X_train = np.append(arr=np.random.rand(X_train.shape[0],npad,X_train.shape[2]),values=X_train,axis=1)
randompadded_X_test = np.append(arr=np.random.rand(X_test.shape[0],npad,X_test.shape[2]),values=X_test,axis=1)

randompadded_model = DualOutputRNN(n_classes=y_train.shape[1],
                      batch_size=10,
                      ts_size=randompadded_X_train.shape[1],
                      epochs=200,
                      earliness_factor=.01,
                      lr=.001,
                      reg=.01)

randompadded_model.fit(randompadded_X_train, y_train)

Epoch: 0001 cost=1.703382659
Epoch: 0011 cost=1.399632370
Epoch: 0021 cost=1.430562007
Epoch: 0031 cost=1.407640433
Epoch: 0041 cost=1.413652205
Epoch: 0051 cost=1.252547884
Epoch: 0061 cost=0.750434625
Epoch: 0071 cost=1.161399627
Epoch: 0081 cost=0.918587714
Epoch: 0091 cost=0.744751233
[End of no-early training] Epoch: 0100 cost=0.680211473
Epoch: 0101 cost=1.784592879
Epoch: 0111 cost=1.472834218
Epoch: 0121 cost=1.012263888
Epoch: 0131 cost=0.799825913
Epoch: 0141 cost=0.734218782
Epoch: 0151 cost=0.776022094
Epoch: 0161 cost=0.655704981
Epoch: 0171 cost=0.697451156
Epoch: 0181 cost=0.595101678
Epoch: 0191 cost=0.555443564


In [13]:
y_pred, tau_pred = randompadded_model.predict(randompadded_X_test)
for yi, yi_hat, taui in zip(y_test, y_pred, tau_pred):
    print(yi, yi_hat, taui)

3 4 11
1 1 16
3 4 11
2 1 17
1 1 17
3 4 11
2 1 17
2 2 16
2 1 18
4 4 11
3 4 11
2 2 17
3 4 11
3 4 11
3 4 11
4 4 11
2 2 16
3 4 11
1 1 17
1 1 16
3 4 11
4 3 10
4 4 11
1 1 16
2 2 17
3 4 11
3 4 11
4 4 12
3 3 10
4 4 11
3 4 11
2 2 17
2 1 17
2 1 17
1 1 17
4 3 11
3 4 11
1 1 16
2 1 18
4 4 12
3 4 11
4 3 10
2 2 17
2 2 17
4 4 11
2 1 17
1 1 17
4 3 10
4 4 10
3 3 10
2 2 18
4 4 11
2 2 17
1 1 17
3 4 11
1 1 16
3 3 10
1 1 17
3 4 10
3 4 11
3 4 11
1 1 16
3 4 11
2 2 18
4 4 11
1 1 17
2 1 17
1 1 16
2 1 16
1 1 16
1 1 16
2 2 17
3 3 10
4 4 11
1 1 15
2 2 17
2 2 18
4 4 11
3 4 11
2 1 17
2 1 16
1 1 15
4 4 12
1 1 17
2 1 17
2 1 16
4 4 11
1 1 17
1 1 17
1 1 15
3 4 11
1 4 0
1 1 18
2 1 17
3 4 11
3 4 11
2 1 16
2 2 16
3 4 11
4 4 11


### 2. Insert a new class that always has the same representation through time

For this class the optimal time of classification should always be the first one since other time observations do not bring further information about this class

In [14]:
np.random.seed(0)

def insert_prototype_as_new_class_to_dataset(X, y, prototype_array, ncopies):
    """
    Insert copies of a prototype timeseries in into the dataset as new class
    """
    
    # repeat and append the prototype array at the beginning of X
    new_samples = np.repeat(prototype_array,ncopies,axis=0)
    X_ = np.append(arr=new_samples,values=X,axis=0)
    
    # extend the old one hot class labels by zeros for the new class
    y_extended = np.pad(y_train,((0,0),(0,1)),mode='constant', constant_values=0)
    
    # add new one hot classes with ones at new class
    y_newclass = np.pad(np.zeros([ncopies,y_train.shape[1]]),((0,0),(0,1)),mode='constant',constant_values=1)
    
    # append the new class labels at the beginning of y
    y_ = np.append(arr=y_newclass,values=y_extended,axis=0)
    
    # shuffle X_ and y_ in unison
    randomize = np.arange(y_.shape[0])
    np.random.shuffle(randomize)
    X_ = X_[randomize]
    y_ = y_[randomize]

    return X_, y_

prototype_array = np.random.rand(1,X_train.shape[1],1)
X_train_aug, y_train_aug = insert_prototype_as_new_class_to_dataset(X_train, y_train, prototype_array, ncopies=25)
X_test_aug, y_test_aug = insert_prototype_as_new_class_to_dataset(X_test, y_test, prototype_array, ncopies=25)

In [15]:
tf.set_random_seed(0)
np.random.seed(0)

aug_model = DualOutputRNN(n_classes=y_train_aug.shape[1],
                      batch_size=10,
                      ts_size=X_train_aug.shape[1],
                      epochs=200,
                      earliness_factor=.01,
                      lr=.001,
                      reg=.01)
aug_model.fit(X_train_aug, y_train_aug)

Epoch: 0001 cost=2.108022869
Epoch: 0011 cost=1.329974145
Epoch: 0021 cost=1.350204537
Epoch: 0031 cost=2.176482052
Epoch: 0041 cost=1.242981896
Epoch: 0051 cost=1.037420084
Epoch: 0061 cost=1.123291130
Epoch: 0071 cost=0.634279519
Epoch: 0081 cost=0.596615739
Epoch: 0091 cost=0.589000729
[End of no-early training] Epoch: 0100 cost=0.593959076
Epoch: 0101 cost=1.828648200
Epoch: 0111 cost=1.102960249
Epoch: 0121 cost=1.039700856
Epoch: 0131 cost=1.029792562
Epoch: 0141 cost=0.959133943
Epoch: 0151 cost=0.964801495
Epoch: 0161 cost=0.922252913
Epoch: 0171 cost=0.848709136
Epoch: 0181 cost=0.749880051
Epoch: 0191 cost=0.800645058


In [25]:
y_pred, tau_pred = model.predict(X_test_aug)
for yi, yi_hat, taui in zip(np.argmax(y_test_aug, axis=1), y_pred, tau_pred):
    print(yi, yi_hat, taui)

2 1 7
4 2 6
2 2 5
3 2 5
1 2 5
4 1 6
2 1 6
5 1 2
2 1 6
4 4 0
1 2 5
5 1 6
4 1 6
2 1 6
3 1 6
3 4 0
4 2 4
3 4 1
3 4 0
2 4 2
2 4 0
4 1 6
1 4 0
3 2 6
1 2 6
4 4 0
4 4 0
1 1 6
5 1 6
1 2 6
5 1 7
5 1 7
1 2 5
5 1 6
3 2 6
1 1 6
1 4 2
5 1 7
1 2 6
5 1 6
3 4 0
5 1 6
4 2 5
1 4 0
5 1 6
4 4 0
5 1 6
3 2 5
2 4 0
2 2 6
4 4 1
5 1 6
3 1 7
3 1 7
3 2 5
1 4 0
5 1 6
3 2 6
5 1 7
5 1 6
2 4 0
4 4 0
2 4 0
2 1 6
5 1 6
4 4 0
1 4 0
1 4 1
1 2 6
4 4 0
1 1 6
4 4 0
5 1 7
1 2 6
1 2 5
2 4 0
4 2 6
3 4 0
1 4 1
3 4 0
5 1 7
4 1 5
2 4 0
3 4 0
3 2 6
5 1 5
4 1 6
2 4 0
5 1 6
4 1 0
5 1 6
4 2 5
5 1 6
5 1 6
3 4 1
1 4 0
1 4 1
2 1 5
4 4 1
4 4 0
1 4 0
3 1 6
4 2 5
3 4 0
5 1 7
2 1 0
1 4 1
2 1 6
4 4 0
4 4 0
1 4 0
4 4 0
3 2 6
4 4 0
2 2 5
1 1 7
2 1 6
5 1 6
2 1 6
1 2 6
4 2 6
3 4 1
4 4 0
4 1 6
4 4 0


In [24]:
.shape

(125,)

In [20]:
y_test

array([3, 1, 3, 2, 1, 3, 2, 2, 2, 4, 3, 2, 3, 3, 3, 4, 2, 3, 1, 1, 3, 4,
       4, 1, 2, 3, 3, 4, 3, 4, 3, 2, 2, 2, 1, 4, 3, 1, 2, 4, 3, 4, 2, 2,
       4, 2, 1, 4, 4, 3, 2, 4, 2, 1, 3, 1, 3, 1, 3, 3, 3, 1, 3, 2, 4, 1,
       2, 1, 2, 1, 1, 2, 3, 4, 1, 2, 2, 4, 3, 2, 2, 1, 4, 1, 2, 2, 4, 1,
       1, 1, 3, 1, 1, 2, 3, 3, 2, 2, 3, 4])